In [170]:
import pandas as pd


# Function Definition

#### Groups the data in a DataFrame by the group_column and concatenates all data in target_column for each group.

#### All other columns are set to their values in the first record of the group. 

#### Can also choose whether to aggregate all records for each group or only the first or last sessions. 

#### Similar to GROUP BY group_column + GROUP_CONCAT(target_column) in SQL.


In [175]:
import pandas as pd

def group_by(data, group_column='user_id', target_column='user_journey', sessions='all', count_from='last'):
   
    # Argument validation
    if not isinstance(data, pd.DataFrame):
        raise TypeError("'data' should be of type 'pandas.DataFrame': received {} instead".format(type(data)))
    if not isinstance(group_column, str):
        raise TypeError("'group_column' should be of type 'str': received {} instead".format(type(group_column)))
    if not isinstance(target_column, str):
        raise TypeError("'target_column' should be of type 'str': received {} instead".format(type(target_column)))
    if not (isinstance(sessions, str) or isinstance(sessions, int)):
        raise TypeError("'sessions' should be of type 'str' or 'int': received {} instead".format(type(sessions)))
    if not isinstance(count_from, str):
        raise TypeError("'count_from' should be of type 'str': received {} instead".format(type(count_from)))
    if group_column not in data.columns:
        raise ValueError("No column named '{}' in the data".format(group_column))
    if target_column not in data.columns:
        raise ValueError("No column named '{}' in the data".format(target_column))
    if isinstance(sessions, str) and sessions not in ["all", "all_except_last"]:
        raise ValueError("Possible 'session' string values are: 'all' and 'all_except_last'")
    if isinstance(sessions, int) and sessions < 0:
        raise ValueError("Sessions must be a non-negative integer")
    if count_from not in ["last", "first"]:
        raise ValueError("Possible 'count_from' values are: 'last' and 'first'")
    
    # Determine the slice indices based on sessions and count_from
    if sessions == "all":
        start, end = 0, None
    elif sessions == "all_except_last":
        start, end = 0, -1
    elif count_from == "last":
        start, end = -sessions, None
    else:  # count_from == "first"
        start, end = 0, sessions

    # Collect rows in a list and then create a DataFrame from this list
    grouped_rows = []
    
    # Get unique group values
    for group_value in data[group_column].unique():
        group_data = data[data[group_column] == group_value]
        user_journey = "-".join(group_data[target_column].iloc[start:end])
        
        # Create a new row with aggregated user_journey
        new_row = group_data.iloc[0].copy()  # copy the first row in the group
        new_row[target_column] = user_journey  # update target_column with concatenated journey
        grouped_rows.append(new_row)  # add the row to the list
    
    # Create DataFrame from list of rows
    df = pd.DataFrame(grouped_rows)
    df.sort_values(by=[group_column], ignore_index=True, inplace=True)
    df.reset_index(drop=True, inplace=True)
    
    return df


#### Returns a new DataFrame where the provided list of pages is to be removed from the user journey strings.

In [189]:
def remove_pages(data, pages = [], target_column = 'user_journey'):
   
    # Check for correct arguments' type
    if not isinstance(data, pd.DataFrame):
        raise TypeError("'data' should be of type 'pandas.DataFrame': received {} instead".format(type(data)))
    
    if not (isinstance(pages, list) or isinstance(pages, set)):
        raise TypeError("'pages' should be of type 'list' or 'set': received {} instead".format(type(pages)))
    
    for item in pages:
        if not isinstance(item, str):
            raise TypeError("'pages' should be a list of 'str': received list of {} instead".format(type(item)))
    
    if not isinstance(target_column, str):
        raise TypeError("'target_column' should be of type 'str': received {} instead".format(type(target_column)))
    
    
    # Check for correct arguments' value
    if target_column not in data.columns:
        raise ValueError("No column named '{}' in the data".format(target_column))
    
    
    
    ### MAIN FUNCTION BODY ###
    
    
    
    # Make a copy of the dataframe as to not accidentaly modify the original data
    df = data.copy()
    pages = set(pages)
    
    if len(pages) == 0:
        return df
    
    # Obtain a list of all user journey strings -> ["page1-page2-...pageN", ...]
    user_journey = list(df[target_column])
    
    # Split the journey strings into pages -> [["page1", "page2", ..."pageN"], ...]
    user_journey = [journey.split('-') for journey in user_journey]
    
    # Include only pages that are not contained in the set of pages to remove
    user_journey = [[page for page in journey if page not in pages] for journey in user_journey]
    
    # Combine the pages into strings again -> ["page1-page2-...pageN", ...]
    user_journey = ["-".join(i) for i in user_journey]
    
    
    # Update the user journey column with the new strings
    df[target_column] = user_journey
    
    
    
    return df
    

#### Returns a new DataFrame where consecutive page duplicates are removed from the user journey strings. Note that if the page duplicates are not consecutive, they are not removed.

In [190]:
def remove_page_duplicates(data, target_column = 'user_journey'):
   
    
    # Check for correct arguments' type
    if not isinstance(data, pd.DataFrame):
        raise TypeError("'data' should be of type 'pandas.DataFrame': received {} instead".format(type(data)))
    
    if not isinstance(target_column, str):
        raise TypeError("'target_column' should be of type 'str': received {} instead".format(type(target_column)))
    
    
    # Check for correct arguments' value
    if target_column not in data.columns:
        raise ValueError("No column named '{}' in the data".format(target_column))
    
    
    
    ### MAIN FUNCTION BODY ###
    
    
    
    # Make a copy of the dataframe as to not accidentaly modify the original data
    df = data.copy()
    
    
    # Obtain a list of all user journey strings -> ["page1-page2-...pageN", ...]
    user_journey = list(df[target_column])
    
    # Split the journey strings into pages -> [["page1", "page2", ..."pageN"], ...]
    user_journey = [journey.split('-') for journey in user_journey]
    
    
    # Loop through all pages and remove consecutive duplicates
    for i in range(len(user_journey)):
        j = 1
        
        while(j < len(user_journey[i])):
            
            if user_journey[i][j-1] == user_journey[i][j]: # Check and remove the page if the next one is the same
                user_journey[i].pop(j)
            
            else:
                j += 1 # Move to the next page
    
    
    # Combine the pages into strings again -> ["page1-page2-...pageN", ...]
    user_journey = ["-".join(i) for i in user_journey]
    
    
    # Update the user journey column with the new strings
    df[target_column] = user_journey
    
    
    
    return df
    

## Main

In [200]:
file_path = r'project-files\user_journey_raw.csv';
df= pd.read_csv(file_path);

In [201]:
df.head()

,user_id,session_id,subscription_type,user_journey
0,1516,2980231,Annual,Homepage-Log in-Log in-Log in-Log in-Log in-Lo...
1,1516,2980248,Annual,Other-Sign up-Sign up-Sign up-Sign up-Sign up-...
2,1516,2992252,Annual,Log in-Log in-Log in-Log in-Log in-Log in
3,1516,3070491,Annual,Homepage-Log in-Log in-Log in-Log in-Log in-Lo...
4,1516,3709807,Annual,Log in-Log in-Log in-Log in-Log in-Log in-Log ...


In [202]:
# Create a copy of the raw data that will be updated and drop the "session_id" column as we don't need it
clean_data = df.copy().drop("session_id", axis=1)
clean_data

,user_id,subscription_type,user_journey
0,1516,Annual,Homepage-Log in-Log in-Log in-Log in-Log in-Lo...
1,1516,Annual,Other-Sign up-Sign up-Sign up-Sign up-Sign up-...
2,1516,Annual,Log in-Log in-Log in-Log in-Log in-Log in
3,1516,Annual,Homepage-Log in-Log in-Log in-Log in-Log in-Lo...
4,1516,Annual,Log in-Log in-Log in-Log in-Log in-Log in-Log ...
...,...,...,...
9930,509095,Annual,Other-Other-Other-Other-Other-Other-Other-Othe...
9931,509095,Annual,Other-Other-Other-Other-Other-Other-Other-Othe...
9932,509095,Annual,Other-Other
9933,509095,Annual,Other-Other-Other-Other-Other-Other-Other-Othe...


In [197]:
# Preprocessing of the data
clean_data = group_by(clean_data)
clean_data = remove_pages(clean_data, [])
clean_data = remove_page_duplicates(clean_data)

In [198]:
clean_data.head()

,user_id,subscription_type,user_journey
0,1516,Annual,Homepage-Log in-Other-Sign up-Log in-Homepage-...
1,3395,Annual,Other-Pricing-Sign up-Log in-Homepage-Pricing-...
2,10107,Annual,Homepage-Career tracks-Homepage-Career tracks-...
3,11145,Monthly,Homepage-Log in-Homepage-Log in-Homepage-Log i...
4,12400,Monthly,Homepage-Career tracks-Sign up-Log in-Other-Ca...


# Save the preprocessed data


In [199]:
clean_data.to_csv('cleaned_user_journey/user_journey.csv', index = False)